In [1]:
import sys
sys.path.append("..")

In [2]:
import os
from langchain_community.graphs import Neo4jGraph

os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "2wsx3edc"

database = os.environ.get('NEO4J_DATABASE')
graph = Neo4jGraph(database=database)

In [3]:
graph.query('MATCH (n) DETACH DELETE n;')
# graph.query('drop index chunk_index;')

[]

In [4]:
import os

from langchain_experimental.graph_transformers import LLMGraphTransformer
# from langchain_openai import ChatOpenAI
from langchain_openai import AzureChatOpenAI
from langchain_experimental.llms.ollama_functions import OllamaFunctions
llm = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    temperature=0
)

In [5]:
# from langchain_experimental.llms.ollama_functions import OllamaFunctions
# model_name = 'qwen2:72b-instruct-q8_0'
# llm = OllamaFunctions(model=model_name, temperature=0)
# llm

In [6]:

# from langchain_community.document_loaders import PyPDFLoader
from tools.PDFTablePyPlumberLoader import PDFTablePyPlumberLoader
pages = []
dir = '../data'
for filename in os.listdir(dir):
    if filename.endswith(".pdf"):
        if '個人保險首續期繳費暨保單貸款還款息作業手冊(113年7月版)_業務通路版_p3_p7' not in filename:
            continue
        loader = PDFTablePyPlumberLoader(os.path.join(dir, filename), llm)
        pages += [loader.load()]
    # break # too much time, only load one file
pages
# import json
# with open('data_read_from_pdf.json', 'w') as file:
#     # page_json = 
#     json.dump(pages, file, indent=4, ensure_ascii=False)

Processing documents: 100%|██████████| 10/10 [00:14<00:00,  1.48s/it]


[[Document(metadata={'source': '個人保險首續期繳費暨保單貸款還款息作業手冊(113年7月版)_業務通路版_p3_p7.pdf', 'page_number': 1}, page_content='回目錄\n目錄\n壹、 繳交保險費應注意事項 ......................................................................................................................... 1\n貳、 繳款人相關規定 ..................................................................................................................................... 3\n一、 繳款人身分 ........................................................................................................................................................... 3\n二、 繳費方式及受理應檢附文件 ................................................................................................................................. 3\n三、 自動轉帳及信用卡付款授權書作業規範【加強身分驗證機制】 ........................................................................ 4\n參、 首期保險費繳交方式及相關規定 ......................................................................................................... 6\n一、 首期保險費-採金融機構轉帳作業 ....................

In [7]:
from tools.graph_builder import TwlfGraphBuilder

graph_builder = TwlfGraphBuilder(graph, max_thread=5)
chunks, root_graph_document, root_document, root_node = graph_builder.build_chunk_graph_with_parent_child(pages)
chunks

[{'chunk_id': '9a1a83ce-9bf4-412f-b6be-454b90c790c5',
  'chunk_doc': Document(metadata={'source': '個人保險首續期繳費暨保單貸款還款息作業手冊(113年7月版)_業務通路版_p3_p7.pdf', 'total_page_num': 5}, page_content='回目錄\n目錄\n壹、 繳交保險費應注意事項 . 1\n貳、 繳款人相關規定 . 3\n一、 繳款人身分 .. 3\n二、 繳費方式及受理應檢附文件  3\n三、 自動轉帳及信用卡付款授權書作業規範【加強身分驗證機制】  4\n參、 首期保險費繳交方式及相關規定  6\n一、 首期保險費-採金融機構轉帳作業  6\n二、 首期保險費-採信用卡扣款作業  7\n三、 首期保險費-採自行匯款作業 . 8\n四、 首期保險費-採自行匯款管道 . 8\n(一) 台幣-首期匯款帳號【業務通路適用】  8\n(二) 外幣-首期匯款帳號')},
 {'chunk_id': '6b2f2fdf-6730-461b-9af7-b5632ef758fa',
  'chunk_doc': Document(metadata={'source': '個人保險首續期繳費暨保單貸款還款息作業手冊(113年7月版)_業務通路版_p3_p7.pdf', 'total_page_num': 5}, page_content='【業務通路適用】  9\n(三) WEB立即繳費 . 10\n(四) 中信ATM繳費(操作方式詳官網) .. 10\n肆、 續期保險費繳交方式及相關規定 . 11\n一、 續期保險費-採金融機構轉帳作業 .. 11\n二、 續期保險費-採信用卡扣款作業 .. 13\n三、 續期保險費-採自行繳費作業  15\n四、 高齡關懷電訪作業 .. 17\n五、 續期保險費扣款不成通知方式 . 18\n伍、 各金融機構核印、扣款時間及相關規定 . 21\n陸、 保單貸款還款息 .. 23\n一、 保單貸款之繳息通知 . 23\n二、 保單借款之還款息帳號  24\n柒、 行動保全續期保費服務說明(僅適用業務通路)  25\n版次：113年07月版 修訂日期：113.07.01\n回目錄\n壹、 繳交保險費應注意事項\n一、 因

In [8]:
graph_builder.graph.add_graph_documents([root_graph_document])
graph_builder.update_node_properties(root_node.id, root_document.metadata)

[{'n': {'file_path': '個人保險首續期繳費暨保單貸款還款息作業手冊(113年7月版)_業務通路版_p3_p7.pdf',
   'filename': '個人保險首續期繳費暨保單貸款還款息作業手冊(113年7月版)_業務通路版_p3_p7.pdf',
   'total_page_num': 5,
   'id': '191baf32-16f1-4522-a1a1-504edf0f2825'}}]

In [8]:
# from tools.TokenCounter import num_tokens_from_string

# tokens_num = 0
# for chunk in chunk_list:
#     doc = chunk['chunk_doc']
#     content = doc.page_content
#     curr_num = num_tokens_from_string(content)
#     tokens_num += num_tokens_from_string(content)
# tokens_num

In [9]:
graph_builder._get_combined_chunks(chunks)

[Document(metadata={'source': '個人保險首續期繳費暨保單貸款還款息作業手冊(113年7月版)_業務通路版_p3_p7.pdf', 'total_page_num': 5, 'combined_chunk_ids': ['9a1a83ce-9bf4-412f-b6be-454b90c790c5']}, page_content='回目錄\n目錄\n壹、 繳交保險費應注意事項 . 1\n貳、 繳款人相關規定 . 3\n一、 繳款人身分 .. 3\n二、 繳費方式及受理應檢附文件  3\n三、 自動轉帳及信用卡付款授權書作業規範【加強身分驗證機制】  4\n參、 首期保險費繳交方式及相關規定  6\n一、 首期保險費-採金融機構轉帳作業  6\n二、 首期保險費-採信用卡扣款作業  7\n三、 首期保險費-採自行匯款作業 . 8\n四、 首期保險費-採自行匯款管道 . 8\n(一) 台幣-首期匯款帳號【業務通路適用】  8\n(二) 外幣-首期匯款帳號'),
 Document(metadata={'source': '個人保險首續期繳費暨保單貸款還款息作業手冊(113年7月版)_業務通路版_p3_p7.pdf', 'total_page_num': 5, 'combined_chunk_ids': ['6b2f2fdf-6730-461b-9af7-b5632ef758fa']}, page_content='【業務通路適用】  9\n(三) WEB立即繳費 . 10\n(四) 中信ATM繳費(操作方式詳官網) .. 10\n肆、 續期保險費繳交方式及相關規定 . 11\n一、 續期保險費-採金融機構轉帳作業 .. 11\n二、 續期保險費-採信用卡扣款作業 .. 13\n三、 續期保險費-採自行繳費作業  15\n四、 高齡關懷電訪作業 .. 17\n五、 續期保險費扣款不成通知方式 . 18\n伍、 各金融機構核印、扣款時間及相關規定 . 21\n陸、 保單貸款還款息 .. 23\n一、 保單貸款之繳息通知 . 23\n二、 保單借款之還款息帳號  24\n柒、 行動保全續期保費服務說明(僅適用業務通路)  25\n版次：113年07月版 修訂日期：113.07.01\n回目錄\n壹、 繳交保險費應注意事項\n一、 因應「保險業保險經紀

In [10]:

graph_documents = graph_builder.get_graph_from_llm(llm, chunks, allowedNodes=[], allowedRelationship=[])


100%|██████████| 24/24 [00:39<00:00,  1.66s/it]


In [11]:
graph_documents[0].source

Document(metadata={'source': '個人保險首續期繳費暨保單貸款還款息作業手冊(113年7月版)_業務通路版_p3_p7.pdf', 'total_page_num': 5, 'combined_chunk_ids': ['b2229051-f17e-4f7f-b0dd-96052f524b89']}, page_content='。首期及\n續期保險費可約定不同繳費方式，續期繳費方式可隨時申請變更，因金融機構轉帳作業須送銀\n行或郵局核印；信用卡扣款作業須完成信用卡身份驗證機制，請於應繳費日前20天申請並檢附\n「保險費暨保險單借款利息自動轉帳付款授權書」/「保險費信用卡付款授權書」。\n三、 繳別：\n(一) 傳統型商品：躉繳、分期繳(月繳、季繳、半年繳、年繳)。\n(二) 投資型商品：躉繳、分期繳(月繳、季繳、半年繳、年繳)、彈性繳(適用於超額保險費)。\n(三) 月繳件首期應繳交2個月保險費(商品或特定通路另有規定者除外)')

In [12]:
graph.add_graph_documents(graph_documents, baseEntityLabel=True)

In [13]:
chunks_and_graphDocuments_list = graph_builder.get_chunk_and_graphDocument(graph_documents)
chunks_and_graphDocuments_list[0]

{'graph_doc': GraphDocument(nodes=[Node(id='首期保險費', type='概念', properties={}), Node(id='續期保險費', type='概念', properties={}), Node(id='金融機構轉帳作業', type='概念', properties={}), Node(id='信用卡扣款作業', type='概念', properties={}), Node(id='傳統型商品', type='概念', properties={}), Node(id='投資型商品', type='概念', properties={}), Node(id='月繳件首期', type='概念', properties={})], relationships=[Relationship(source=Node(id='首期保險費', type='概念', properties={}), target=Node(id='續期保險費', type='概念', properties={}), type='繳費方式', properties={}), Relationship(source=Node(id='續期保險費', type='概念', properties={}), target=Node(id='金融機構轉帳作業', type='概念', properties={}), type='需送核印', properties={}), Relationship(source=Node(id='續期保險費', type='概念', properties={}), target=Node(id='信用卡扣款作業', type='概念', properties={}), type='需身份驗證', properties={}), Relationship(source=Node(id='傳統型商品', type='概念', properties={}), target=Node(id='躉繳', type='概念', properties={}), type='繳別', properties={}), Relationship(source=Node(id='傳統型商品', type='概念', properties=

In [14]:
graph_builder.merge_relationship_between_chunk_and_entites(chunks_and_graphDocuments_list)